In [ ]:
import sys
import os
sys.path.insert(0, "../../python")

import numpy as np
from beam.beams import laserpulse
from beam.beams import laserbeam
from beam.elements import plasma
from beam import interactions
from ionization import ionization
from lens import profile
from lens import bessel
from lens import ray
import matplotlib.pyplot as plt
from unwrap import unwrap

basePath = '/media/robert/Data_Storage/Data/Diagnostic/Phase_Diagnostic/Nominal_Plasma/'

In [ ]:
path = basePath + 'Oblique/1e17_w-10e3_theta-0.859/'
# Build the plasma
plasmaParams = {
    'Nx' : 2**12,
    'Ny' : 2**10,
    'Nz' : 2**8,
    'X' : 60e3,
    'Y' : 60e3,
    'Z' : 2.0e6, 
    'n0' : 1.0,
    'atom' : ionization.Ar,
    'path' : path,
    'name' : 'ArgonPlasma',
    'load' : False,
    'cyl' : True
}

w = 200
z0 = 0.75e6
zf = plasmaParams['Z']
dz = 0.5e6
sigmaIn = 15e4
sigmaOut = 15e4
z, ne = profile.plasma_gaussian_ramps(z0, dz, sigmaIn, sigmaOut, plasmaParams['Nz'], zf)

argon = plasma.Plasma(plasmaParams)
n = np.full((argon.Nx, argon.Ny, argon.Nz), plasmaParams['n0'], dtype='double')
r2 = argon.x[:, None, None]**2 + argon.y[None, :, None]**2
ne = plasmaParams['n0'] * ne[None, None, :] * np.exp(-r2/w**2)
argon.initialize_plasma(n, ne)
del n
del ne
argon.plot_long_density_center()

In [ ]:
# Build the beam
beamParams = {
    'Nx' : plasmaParams['Nx'],
    'Ny' : plasmaParams['Ny'],
    'X' : plasmaParams['X'],
    'Y' : plasmaParams['Y'],
    'lam' : 0.8,
    'path' : path,
    'name' : 'ProbePulse',
    'load' : False,
    'threads' : 4,
    'cyl' : False,
    'E0' : 27.44923, # Gives peak input intensity of 1.0
    'waist' : 10000,
    'z0' : 0.0,
    'theta' : 0.859,
    'dx' : -15e3
}

beam = laserbeam.GeneralGaussianLaser(beamParams)
beam.plot_current_intensity()